In [516]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [517]:
movies = pd.read_csv('../input/disney-movies-and-tv-shows/disney_plus_titles.csv')
movies.info()

In [518]:
movies.head()

In [519]:
required_columns =['type','title', 'rating', 'listed_in', 'description', 'director', 'cast']
movies=movies[required_columns]
movies.head()

In [520]:
movies.isna().sum()

In [521]:
movies.fillna(' ', inplace=True)
print(movies.isna().sum())
print(movies.iloc[0])

In [522]:
movies.info()

In [523]:
movies['listed_in'] = movies['listed_in'].apply(lambda x:[i.replace(" ","") for i in x])
movies['description'] = movies['description'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['director'] = movies['director'].apply(lambda x: x.replace(" ","") )
movies.info()

In [524]:
movies['content'] = movies['title'].astype(str)+' '+movies['rating'].astype(str)+' '+movies['description'].astype(str)+' '+movies['cast'].astype(str)+' '+movies['director'].astype(str)
print(movies['content'])

In [525]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [526]:
movies['cont']=movies['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [527]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
movie_vectors = vectorizer.fit_transform(movies['cont'].values) 

In [528]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(movie_vectors)
similarity_df = pd.DataFrame(similarity)
print(similarity_df)

In [529]:
def recommend(movie):
    # find movie index from dataset
    movies_index = movies[movies['title'] == movie].index[0]
    
    # finding cosine similarities of movie
    distances = similarity[movies_index]
    
    # sorting cosine similarities
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:11]
    l = []
    x = 0
    for i in movies_list:
        l.append(movies_list[x][1])
        print(movies.iloc[i[0]].title, movies_list[x][1])
        x = x + 1
    return l

In [530]:
l = recommend('Ice Age: A Mammoth Christmas')
count = list(range(1, len(l)+1))
plt.plot(count, l, color='red', marker='o')
plt.title('Similarity Scores of Top 10 Recommendations', fontsize=18)
plt.ylabel('Similarity Score', fontsize=14)
plt.grid(True)
plt.show()

In [531]:
print("Mean of the Similarity Scores:")
print(sum(l) / len(l))